bw_sandbox.ipynb
sandbox

In [1]:
import yfinance as yf
import pandas as pd
# show all columns
pd.set_option('display.max_columns', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import openpyxl
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
import os


from typing import Dict, Optional

import requests
from io import StringIO

In [2]:
import asyncio
import aiohttp

In [3]:
import os


In [4]:
import yfinance as yf
import json
from pathlib import Path
import time


In [5]:
import logging
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# load biotech stock raw data from tradingview
biotech_all_df = pd.read_csv('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/biotech_nev_all_tv_2024-10-28.csv')
# # return column 0, 1, 4, 7
# biotech_all_df = biotech_all_df.iloc[:, [0, 1, 4, 7]]


print(len(biotech_all_df))
biotech_all_df.head()

445


,Symbol,Description,Price,Price - Currency,Industry,Market capitalization,Market capitalization - Currency,Enterprise value,Enterprise value - Currency,Price to sales ratio,"Total revenue, Annual","Total revenue, Annual - Currency","EBITDA, Trailing 12 months","EBITDA, Trailing 12 months - Currency","Revenue per employee, Annual","Revenue per employee, Annual - Currency"
0,AMGN,Amgen Inc.,317.17,USD,Biotechnology,1.704247e+11,USD,2.236666e+11,USD,5.579327,2.801000e+10,USD,1.157000e+10,USD,1.049064e+06,USD
1,GILD,"Gilead Sciences, Inc.",88.76,USD,Biotechnology,1.105055e+11,USD,1.298723e+11,USD,4.008316,2.705800e+10,USD,1.287700e+10,USD,1.503222e+06,USD
2,CSLLY,CSL Ltd.,96.41,USD,Biotechnology,9.389904e+10,USD,1.064630e+11,USD,6.450868,1.469000e+10,USD,4.632880e+09,USD,4.590625e+05,USD
3,CMXHF,CSL Ltd.,190.25,USD,Biotechnology,9.212030e+10,USD,1.064395e+11,USD,6.283493,1.494359e+10,USD,4.726053e+09,USD,4.669872e+05,USD
4,LZAGY,Lonza Group AG,63.56,USD,Biotechnology,4.620749e+10,USD,4.825623e+10,USD,5.831281,7.474531e+09,USD,2.325579e+09,USD,4.152517e+05,USD


In [8]:
# filter for Enterprise Value < 0, creates NEV dataframe
bt_nev_df = biotech_all_df[biotech_all_df['Enterprise value'] < 0]
print(len(bt_nev_df))
# sort descending by Enterprise value
bt_nev_df = bt_nev_df.sort_values(by='Symbol', ascending=True)
print(len(bt_nev_df))
# filter out rows wherre Total Revenue is 0
bt_nev_df = bt_nev_df[bt_nev_df['Total revenue, Annual'] > 0]
print(len(bt_nev_df))
bt_nev_df = bt_nev_df[bt_nev_df['Revenue per employee, Annual'] > 0]
print(len(bt_nev_df))


# bt_nev_df.head()

62
62
26
24


In [ ]:
"""
remove shell companies.
Shell companies are companies that are not operational and have no business activity
signs of a shell company:
- no revenue
- no employees
- no physical address
- no products or services
- no customers
- no assets


also avoid 
debt to asset ratio > .5 
current ratio < 1 ??? what is this
""" 


In [8]:
# create a list 'bt_nev_ticker_list' 
bt_nev_list = bt_nev_df['Symbol'].tolist()


print(len(bt_nev_list)) 
print(bt_nev_list)

24
['AADI', 'ALEC', 'APDN', 'ASCLF', 'CODX', 'CRBU', 'DTIL', 'FDMT', 'GLPG', 'HOOK', 'IKNA', 'IPSC', 'ITOS', 'KZR', 'LVTX', 'LYEL', 'SABS', 'SEER', 'SPRB', 'STTK', 'SYBX', 'TSBX', 'VIR', 'VTGN']


remember you still have yfinance in the arsenal here. use it for better screening.
screening is important if not crucial.
patience. 


In [14]:
# # ------------------------------------------------
# # which these are potential shells? -> activate yahoo finance
# # ------------------------------------------------
# # get stock data from yahoo finance
# def get_stock_data(ticker):
#     try:
#         stock = yf.Ticker(ticker)
#         hist = stock.history(period="1y")
#         info = stock.info
#         return ticker, hist, info
#     except:
#         return ticker, None, None

# ticker hack 1 - start

In [39]:
####----> 0
########################################
# --- Input variables --- #
# working
ticker_hack = 'ATNM'

########################################
####----> 1
# get stock data from yahoo finance
def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")
        info = stock.info
        return ticker, hist, info
    except:
        return ticker, None, None
    

####----> 2
#### 
# return stock data for the stock ticker
####
ticker, hist, info = get_stock_data(ticker_hack)
print(ticker)
# print(hist)
# print(info)

# # print info but beutify? it
# for key, value in info.items():
#     # print info in a pretty JSON format
#     print(json.dumps(info, indent=4))   


####----> 3
def get_latest_stock_data(ticker):
    """
    Fetches the most recent historical data and info for a given ticker.
    
    Parameters:
        ticker (str): The stock ticker symbol.
        
    Returns:
        tuple: (ticker, latest_hist, info)
            - ticker (str): The stock ticker.
            - latest_hist (dict): The most recent historical data point.
            - info (dict): Informational data about the stock.
    """
    try:
        ticker_obj = yf.Ticker(ticker)
        
        # Fetch the most recent historical data (last 1 day)
        hist = ticker_obj.history(period='1d', interval='5m')  # 1-minute interval for the latest day
        
        if hist.empty:
            logging.warning(f"No historical data found for ticker {ticker}.")
            latest_hist = {}
        else:
            # Get the last available row
            latest_row = hist.tail(1)
            latest_hist = latest_row.to_dict(orient='records')[0]
        
        # Fetch the info dictionary
        info = ticker_obj.info
        
        return ticker, latest_hist, info

    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return ticker, {}, {}


####----> 4
# ### nested function that saves output as json to corresponding folder at path /Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/bio_targets/{ticker}.json
# def save_to_json(ticker, data):
#     # Define the path
#     path = f'/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/bio_targets/{ticker}/yahoo_finance_new_data_{ticker}.json'
    
#     # Save data as JSON
#     with open(path, 'w') as f:
#         json.dump(data, f, indent=4)

# # Prepare data to save
# data_to_save = {
#     'ticker': ticker,
#     # 'latest_hist': latest_hist,
#     'info': info
# }

# # Call the nested function to save data
# save_to_json(ticker, data_to_save)

#############################################
def save_to_json(ticker, data):
    # Define the path
    directory = f'/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/bio_targets/{ticker}'
    path = f'{directory}/yahoo_finance_new_data_{ticker}.json'
    
    # Create the directory if it doesn't exist
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    # Save data as JSON
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)

# Prepare data to save
data_to_save = {
    'ticker': ticker,
    'info': info
}

# Call the function to save data
save_to_json(ticker, data_to_save)
########################################

####----> 5
if ticker_hack == ticker:
    print(True)
else:
    print(False)

ATNM
True


In [34]:
# --- missing retrival --- #
# ticker_hack = 'ANTM'
# ticker_hack = 'PRE'
# ticker_hack = 'STTX'

missing_tickers = ['ANTM', 'PRE', 'STTX']

In [ ]:
# ###----> 6
# get_latest_stock_data(ticker)

 # END of json most recent data loads from yahoo finance

start bloomberg import
test -> compare output

In [9]:
# -------------
# load SB's bloomberg data to df
# -------------
# load to dataframe .xlsx
bb_df = pd.read_excel('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/Alternative EV Screen.xlsx', sheet_name='Cleaned')


print(len(bb_df))
# bb_df.tail(10)
bb_df.head(10)

52


/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/sudz4/Desktop/SPS_local/sps/sps_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,Ticker,Last Price,YTD Returns,Cover Days,SI,NT Uptrend,Intermediate Term Uptrend,20 DMA,50 DMA,100 DMA,200 DMA,2Q24,3Q24,4Q24,ADV $M,Cash Runway (Qs),Market Cap,Total Debt,Current,2Q24 EV,3Q24 EV,Cash on Hand,Net Loss (Qs),20 DMA.1,50 DMA.1,100 DMA.1,200 DMA.1
0,PRE,4.7700,-0.192893,1.056601,0.003922,-0.010658,-0.074336,0.035944,0.024903,-0.089601,-0.051284,2.166148,1.965543,1.764937,100697.085,15.988999,57.326467,2.369388,-124.177628,-112.677628,-101.177628,183.873483,-11.500,4.604495,4.654098,5.239461,5.027851
1,ANTX,1.1000,-0.946315,1.365465,0.006300,-0.007799,-0.729220,0.041667,0.033543,-0.334483,-0.720137,2.184605,1.834280,1.483956,180061.2,9.090435,32.826674,0.000000,-71.713326,-60.213326,-48.713326,104.540000,-11.500,1.056,1.064300,1.652850,3.930500
2,BOLT,0.6596,-0.411039,1.523604,0.003992,-0.000992,-0.282287,0.019971,0.018959,-0.044480,-0.268680,2.247511,1.714566,1.181621,84387.276119,7.596566,25.240868,20.219000,-56.729132,-43.277132,-29.825132,102.189000,-13.452,0.646685,0.647327,0.690304,0.901930
3,ALGS,8.7500,-0.472892,1.754932,0.033707,-0.262578,-0.327272,0.128086,-0.168124,-0.230097,-0.440374,1.982132,1.647975,1.313818,859174.326172,10.116212,27.965903,11.138000,-55.432097,-46.087097,-36.742097,94.536000,-9.345,7.7565,10.518400,11.365075,15.635444
4,KZR,0.7651,-0.192421,0.671468,0.017282,0.191504,-0.093192,-0.061710,0.117976,0.164678,0.013790,1.601938,1.189924,0.777911,826851.412275,7.138348,55.823395,18.933000,-89.425605,-66.425605,-43.425605,164.182000,-23.000,0.81542,0.684362,0.656920,0.754693
5,PMVP,1.6535,-0.466613,5.534643,0.019480,0.010421,-0.082081,0.062490,0.073562,0.055302,-0.014557,1.343443,1.126745,0.910048,315647.517766,11.534074,85.187908,13.286000,-114.445092,-95.985092,-77.525092,212.919000,-18.460,1.55625,1.540200,1.566850,1.677925
6,ELEV,0.5901,0.098883,11.328960,0.105990,-0.107063,-0.764517,0.058988,-0.054391,-0.615907,-0.777325,1.314062,0.981483,0.648904,416232.087731,9.555948,34.878928,30.137000,-45.833072,-34.233072,-22.633072,110.849000,-11.600,0.55723,0.624042,1.536345,2.650048
7,SEER,1.9750,0.018041,3.114457,0.016033,0.077928,-0.012663,0.022786,0.102490,0.105885,0.088529,1.103434,0.948475,0.793516,404392.513672,15.077287,121.322557,28.259000,-133.871443,-115.071443,-96.271443,283.453000,-18.800,1.931,1.791400,1.785900,1.814375
8,HLVX,1.8700,-0.883489,8.282363,0.045154,-0.005504,-0.810772,0.034864,0.029169,-0.557092,-0.805252,1.082186,0.868432,0.654679,258154.610313,12.313568,93.097834,51.193000,-100.749166,-80.849166,-60.949166,245.040000,-19.900,1.807,1.817000,4.222100,9.602175
9,VYNE,2.7700,0.188841,1.043871,0.003942,0.089166,-0.092941,0.289122,0.404067,0.404224,0.273572,0.906386,0.681204,0.456022,278878.882344,8.489239,40.855830,0.214000,-37.031170,-27.831170,-18.631170,78.101000,-9.200,2.14875,1.972840,1.972620,2.174985


In [20]:
# list of tickers from bb 
bb_ticker_list = bb_df.iloc[:47]['Ticker'].tolist()
# # sort the list alphabetically
# bb_ticker_list.sort()

print(len(bb_ticker_list))
print(bb_ticker_list)

47
['PRE ', 'ANTX ', 'BOLT ', 'ALGS ', 'KZR ', 'PMVP ', 'ELEV ', 'SEER ', 'HLVX ', 'VYNE ', 'LVTX ', 'KRON ', 'RLYB ', 'BRNS ', 'ITOS ', 'LYEL ', 'TARA ', 'ZNTL ', 'PRLD ', 'CDTX ', 'CMRX ', 'AVTE ', 'VTYX ', 'CRBU ', 'ALXO ', 'AADI ', 'IKNA ', 'ABOS ', 'PASG ', 'ATNM ', 'STTK ', 'ACRS ', 'EQ ', 'RPID ', 'RAPT ', 'XLO ', 'SCYX ', 'VTGN ', 'ALVR ', 'IPSC ', 'QURE ', 'IVVD ', 'MIST ', 'XFOR ', 'NKTX ', 'IOBT ', 'VERV ']


In [101]:
# # Define the base directory
# base_dir = '/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon'

# # Create the top folder 'bio_targets' if it doesn't exist
# top_folder_path = os.path.join(base_dir, 'bio_targets')
# os.makedirs(top_folder_path, exist_ok=True)

# # Create folders for each stock in the bb_ticker_list inside 'bio_targets'
# for ticker in bb_ticker_list:
#     # Strip any leading/trailing whitespace from the ticker
#     ticker = ticker.strip()
#     # Define the path for the new folder
#     folder_path = os.path.join(top_folder_path, ticker)
#     # Create the folder if it doesn't exist
#     os.makedirs(folder_path, exist_ok=True)

# print("Folders created successfully.")

Folders created successfully.


In [11]:
# print length of folders in bio_targets
tfp = '/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/bio_targets'
len(os.listdir(tfp))

47

In [12]:
# save as list columns of bb_df
bb_df_cols = bb_df.columns.tolist()

print(bb_df_cols)

['Ticker', 'Last Price', 'YTD Returns', 'Cover Days', 'SI', 'NT Uptrend', 'Intermediate Term Uptrend', '20 DMA', '50 DMA', '100 DMA', '200 DMA', '2Q24', '3Q24', '4Q24', 'ADV $M', 'Cash Runway (Qs)', 'Market Cap', 'Total Debt', 'Current', '2Q24 EV', '3Q24 EV', 'Cash on Hand', 'Net Loss (Qs)', '20 DMA.1', '50 DMA.1', '100 DMA.1', '200 DMA.1']


In [21]:
print(bb_ticker_list)

['PRE ', 'ANTX ', 'BOLT ', 'ALGS ', 'KZR ', 'PMVP ', 'ELEV ', 'SEER ', 'HLVX ', 'VYNE ', 'LVTX ', 'KRON ', 'RLYB ', 'BRNS ', 'ITOS ', 'LYEL ', 'TARA ', 'ZNTL ', 'PRLD ', 'CDTX ', 'CMRX ', 'AVTE ', 'VTYX ', 'CRBU ', 'ALXO ', 'AADI ', 'IKNA ', 'ABOS ', 'PASG ', 'ATNM ', 'STTK ', 'ACRS ', 'EQ ', 'RPID ', 'RAPT ', 'XLO ', 'SCYX ', 'VTGN ', 'ALVR ', 'IPSC ', 'QURE ', 'IVVD ', 'MIST ', 'XFOR ', 'NKTX ', 'IOBT ', 'VERV ']


In [44]:
# # Return the bb_ticker_list if not also in bt_nev_list
# unique_bb_tickers = [ticker for ticker in bb_ticker_list if ticker.strip() not in bt_nev_list]

# print(len(unique_bb_tickers))
# print(unique_bb_tickers)

20
['PRE ', 'BOLT ', 'ALGS ', 'VYNE ', 'KRON ', 'BRNS ', 'TARA ', 'CDTX ', 'CMRX ', 'ABOS ', 'ATNM ', 'ACRS ', 'EQ ', 'RPID ', 'RAPT ', 'XLO ', 'SCYX ', 'QURE ', 'MIST ', 'VERV ']


In [97]:
bb_df = bb_df.iloc[:47]


print(len(bb_df))
bb_df

47


,Ticker,Last Price,YTD Returns,Cover Days,SI,NT Uptrend,Intermediate Term Uptrend,20 DMA,50 DMA,100 DMA,200 DMA,2Q24,3Q24,4Q24,ADV $M,Cash Runway (Qs),Market Cap,Total Debt,Current,2Q24 EV,3Q24 EV,Cash on Hand,Net Loss (Qs),20 DMA.1,50 DMA.1,100 DMA.1,200 DMA.1
0,PRE,4.7700,-0.192893,1.056601,0.003922,-0.010658,-0.074336,0.035944,0.024903,-0.089601,-0.051284,2.166148,1.965543,1.764937,100697.085,15.988999,57.326467,2.369388,-124.177628,-112.677628,-101.177628,183.873483,-11.500,4.604495,4.654098,5.239461,5.027851
1,ANTX,1.1000,-0.946315,1.365465,0.006300,-0.007799,-0.729220,0.041667,0.033543,-0.334483,-0.720137,2.184605,1.834280,1.483956,180061.2,9.090435,32.826674,0.000000,-71.713326,-60.213326,-48.713326,104.540000,-11.500,1.056,1.064300,1.652850,3.930500
2,BOLT,0.6596,-0.411039,1.523604,0.003992,-0.000992,-0.282287,0.019971,0.018959,-0.044480,-0.268680,2.247511,1.714566,1.181621,84387.276119,7.596566,25.240868,20.219000,-56.729132,-43.277132,-29.825132,102.189000,-13.452,0.646685,0.647327,0.690304,0.901930
3,ALGS,8.7500,-0.472892,1.754932,0.033707,-0.262578,-0.327272,0.128086,-0.168124,-0.230097,-0.440374,1.982132,1.647975,1.313818,859174.326172,10.116212,27.965903,11.138000,-55.432097,-46.087097,-36.742097,94.536000,-9.345,7.7565,10.518400,11.365075,15.635444
4,KZR,0.7651,-0.192421,0.671468,0.017282,0.191504,-0.093192,-0.061710,0.117976,0.164678,0.013790,1.601938,1.189924,0.777911,826851.412275,7.138348,55.823395,18.933000,-89.425605,-66.425605,-43.425605,164.182000,-23.000,0.81542,0.684362,0.656920,0.754693
5,PMVP,1.6535,-0.466613,5.534643,0.019480,0.010421,-0.082081,0.062490,0.073562,0.055302,-0.014557,1.343443,1.126745,0.910048,315647.517766,11.534074,85.187908,13.286000,-114.445092,-95.985092,-77.525092,212.919000,-18.460,1.55625,1.540200,1.566850,1.677925
6,ELEV,0.5901,0.098883,11.328960,0.105990,-0.107063,-0.764517,0.058988,-0.054391,-0.615907,-0.777325,1.314062,0.981483,0.648904,416232.087731,9.555948,34.878928,30.137000,-45.833072,-34.233072,-22.633072,110.849000,-11.600,0.55723,0.624042,1.536345,2.650048
7,SEER,1.9750,0.018041,3.114457,0.016033,0.077928,-0.012663,0.022786,0.102490,0.105885,0.088529,1.103434,0.948475,0.793516,404392.513672,15.077287,121.322557,28.259000,-133.871443,-115.071443,-96.271443,283.453000,-18.800,1.931,1.791400,1.785900,1.814375
8,HLVX,1.8700,-0.883489,8.282363,0.045154,-0.005504,-0.810772,0.034864,0.029169,-0.557092,-0.805252,1.082186,0.868432,0.654679,258154.610313,12.313568,93.097834,51.193000,-100.749166,-80.849166,-60.949166,245.040000,-19.900,1.807,1.817000,4.222100,9.602175
9,VYNE,2.7700,0.188841,1.043871,0.003942,0.089166,-0.092941,0.289122,0.404067,0.404224,0.273572,0.906386,0.681204,0.456022,278878.882344,8.489239,40.855830,0.214000,-37.031170,-27.831170,-18.631170,78.101000,-9.200,2.14875,1.972840,1.972620,2.174985


historical stock buyouts;
10yr
nasdaq, nyse
biotechnology
NEV

use twitter, x buyout study info to to high publicity and data availability

R and mlr determine p-values (statistical significance)

risk curves, jim simmons, renaissance 

astronomical models, astronomical model incorporation overlay


# scaled yahoo finance most recent data retrieval 
#### for all stocks in biotech target list

In [41]:
print(len(bb_ticker_list))
print(bb_ticker_list)

47
['PRE ', 'ANTX ', 'BOLT ', 'ALGS ', 'KZR ', 'PMVP ', 'ELEV ', 'SEER ', 'HLVX ', 'VYNE ', 'LVTX ', 'KRON ', 'RLYB ', 'BRNS ', 'ITOS ', 'LYEL ', 'TARA ', 'ZNTL ', 'PRLD ', 'CDTX ', 'CMRX ', 'AVTE ', 'VTYX ', 'CRBU ', 'ALXO ', 'AADI ', 'IKNA ', 'ABOS ', 'PASG ', 'ATNM ', 'STTK ', 'ACRS ', 'EQ ', 'RPID ', 'RAPT ', 'XLO ', 'SCYX ', 'VTGN ', 'ALVR ', 'IPSC ', 'QURE ', 'IVVD ', 'MIST ', 'XFOR ', 'NKTX ', 'IOBT ', 'VERV ']


# START -> modular custom dataframes ?

| NOTES |
|-|
| END OF PROGRAM |

In [ ]:
# def fetch_stock_data(ticker="ADMA", period="1d"):
#     """
#     Fetches comprehensive stock data for the specified ticker
    
#     Parameters:
#     ticker (str): Stock ticker symbol
#     period (str): Time period to fetch data for (1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max)
    
#     Returns:
#     tuple: (stock_info, historical_data, financial_data, institutional_holders)
#     """
#     # Create ticker object
#     stock = yf.Ticker(ticker)
    
#     # Get stock info (general company information)
#     stock_info = {
#         'company_name': stock.info.get('longName'),
#         'sector': stock.info.get('sector'),
#         'industry': stock.info.get('industry'),
#         'market_cap': stock.info.get('marketCap'),
#         'beta': stock.info.get('beta'),
#         'forward_pe': stock.info.get('forwardPE'),
#         'trailing_pe': stock.info.get('trailingPE'),
#         'dividend_yield': stock.info.get('dividendYield'),
#         'fifty_two_week_high': stock.info.get('fiftyTwoWeekHigh'),
#         'fifty_two_week_low': stock.info.get('fiftyTwoWeekLow'),
#         'analyst_target_price': stock.info.get('targetMeanPrice'),
#         'recommendation': stock.info.get('recommendationKey')
#     }
    
#     # Get historical data
#     hist_data = stock.history(period=period)
    
#     # Get financial data
#     financial_data = {
#         'balance_sheet': stock.balance_sheet,
#         'income_statement': stock.income_stmt,
#         'cash_flow': stock.cashflow,
#         'earnings': stock.earnings
#     }
    
#     # Get institutional holders
#     institutional_holders = stock.institutional_holders
    
#     # Get options data
#     options = {
#         'available_expirations': stock.options,
#         'current_price': stock.info.get('regularMarketPrice'),
#         'implied_volatility': stock.info.get('impliedVolatility')
#     }
    
#     return {
#         'stock_info': stock_info,
#         'historical_data': hist_data,
#         'financial_data': financial_data,
#         'institutional_holders': institutional_holders,
#         'options_data': options
#     }

# def analyze_stock_metrics(data):
#     """
#     Calculates key technical and statistical metrics
#     """
#     hist_data = data['historical_data']
    
#     # Calculate daily returns
#     hist_data['Returns'] = hist_data['Close'].pct_change()
    
#     # Calculate volatility (20-day rolling)
#     hist_data['Volatility'] = hist_data['Returns'].rolling(window=20).std() * (252 ** 0.5)
    
#     # Calculate moving averages
#     hist_data['MA50'] = hist_data['Close'].rolling(window=50).mean()
#     hist_data['MA200'] = hist_data['Close'].rolling(window=200).mean()
    
#     # Calculate RSI
#     delta = hist_data['Close'].diff()
#     gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
#     loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
#     rs = gain / loss
#     hist_data['RSI'] = 100 - (100 / (1 + rs))
    
#     return hist_data

# # Example usage
# if __name__ == "__main__":
#     # Fetch data
#     adma_data = fetch_stock_data("ADMA")
    
#     # Print basic info
#     print("\nStock Information:")
#     for key, value in adma_data['stock_info'].items():
#         print(f"{key}: {value}")
    
#     # Analyze metrics
#     analysis = analyze_stock_metrics(adma_data)
    
#     print("\nLatest Metrics:")
#     print(f"Current Price: ${analysis['Close'][-1]:.2f}")
#     print(f"50-day MA: ${analysis['MA50'][-1]:.2f}")
#     print(f"200-day MA: ${analysis['MA200'][-1]:.2f}")
#     print(f"RSI: {analysis['RSI'][-1]:.2f}")
#     print(f"Annual Volatility: {analysis['Volatility'][-1]:.2%}")